# Creating the data to test in STATA and Python

In [1]:
import numpy as np
import pandas as pd
from ujive1 import *
from ujive2 import *
from tsls import * 

#Pick a vector length:
n = 1000

#Getting our Z's and making a Z matrix:
Z = np.random.randn(n, 1)
column_of_ones = np.ones((Z.shape[0], 1))
Z = np.hstack((column_of_ones, Z))
#pprint(Z)

#Parameter vectors:
α = np.array([1,1])
β = np.array([1,2])
#pprint(α)
#pprint(β)

#Error terms:
e1 = np.random.normal(0,5,n)
e2 = np.random.normal(0,5,n)
δ = np.random.normal(0,1)
ε = 2*e1 - 2*e2 + δ

#Making our endogenous variable:
x = np.dot(Z,α) + .2*e1
X = np.column_stack((column_of_ones, x))
#pprint(X)

#Outcome vector:
Y = np.dot(X,β) + ε

#OLS benchmark:
bhat_ols = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T, Y))

# Check to see if the Z'Z matrix is invertible
if np.linalg.matrix_rank(Z.T @ Z) == Z.shape[1]:  # Should be True
    print("Z'Z is invertible")
else:
    print("Z'Z is not invertible")

cond_number = np.linalg.cond(Z.T @ Z)
print(f"Condition number of Z.T @ Z: {cond_number}")
if cond_number > 1e10:  # Threshold for ill-conditioning
    raise ValueError("Z.T @ Z is ill-conditioned and may cause numerical instability.")


#2sls comparison:
Zt_Z = np.dot(Z.T, Z)
Zt_Z_inv = np.linalg.inv(Zt_Z)
pz = np.dot(np.dot(Z, Zt_Z_inv), Z.T)
proj_x = np.dot(pz, X)
first = np.linalg.inv(np.dot(proj_x.T, X))
second = np.dot(proj_x.T, Y)
bhat_2sls = np.dot(first, second)


ujive1 = UJIVE1(Y,X,Z,talk=False)
ujive2 = UJIVE2(Y,X,Z,talk=False)
tsls = TSLS(Y,X,Z, talk=False)

# Combine matrices into a single DataFrame
df = pd.DataFrame({
    "Y": Y,  # Outcome vector
    **{f"X{i}": X[:, i] for i in range(X.shape[1])},  # Endogenous variables
    **{f"Z{i}": Z[:, i] for i in range(Z.shape[1])}   # Instrumental variables
})

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

#Compare them:
print("OLS:", bhat_ols[1])
print("2SLS:", bhat_2sls[1])

/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive1.py:186: RuntimeWarning: Normally this estimator is used when Z has more columns than X. In this case Z has 2 columns and X has 2 columns.
  warnings.warn(f"Normally this estimator is used when Z has more columns than X. In this case Z has {Z.shape[1]} columns and X has {X.shape[1]} columns.", RuntimeWarning)
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive1.py:226: RuntimeWarning: divide by zero encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive1.py:226: RuntimeWarning: overflow encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive1.py:226: RuntimeWarning: invalid value encountered in matmul
  P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive1.py:227: RuntimeWarning: divide by zero encountered in matmul
  fit = P @ X #  Z @ np.li

Z'Z is invertible
Condition number of Z.T @ Z: 1.1087912177777157
OLS: 7.2906150687927385
2SLS: 2.3361079810448113


/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:217: RuntimeWarning: divide by zero encountered in matmul
  fit = Z @ np.linalg.inv(Z.T @ Z) @ Z.T @ X
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:217: RuntimeWarning: overflow encountered in matmul
  fit = Z @ np.linalg.inv(Z.T @ Z) @ Z.T @ X
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:217: RuntimeWarning: invalid value encountered in matmul
  fit = Z @ np.linalg.inv(Z.T @ Z) @ Z.T @ X
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:220: RuntimeWarning: divide by zero encountered in matmul
  leverage = np.diag(Z @ np.linalg.inv(Z.T @ Z) @ Z.T)
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:220: RuntimeWarning: overflow encountered in matmul
  leverage = np.diag(Z @ np.linalg.inv(Z.T @ Z) @ Z.T)
/Users/jonathanhyatt/weak_instruments/weak_instruments/ujive2.py:220: RuntimeWarning: invalid value encountered in matmul
  leverage = np.diag(Z @ np.lin

In [2]:
print(ujive1.summary())


UJIVE1 Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    0.393414    0.621928 0.632571 0.527159       -0.827023         1.613851
    2.316743    0.447349 5.178821 0.000000        1.438889         3.194596
--------------------------------------------------------------------------------
R-squared: 0.223025
Adjusted R-squared: 0.222247
F-statistic: 54.113603
Root MSE: 13.940452
None


In [3]:
print(ujive2.summary())


UJIVE2 Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    0.394608    0.621867 0.634554 0.525865       -0.825709         1.614924
    2.315483    0.447244 5.177228 0.000000        1.437837         3.193130
--------------------------------------------------------------------------------
R-squared: 0.222927
Adjusted R-squared: 0.222148
F-statistic: 54.047945
Root MSE: 13.941335
None


In [4]:
print(tsls.summary())


TSLS Regression Results
 Coefficient  Std. Error   t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
    0.375053    0.620771 0.604173 0.545866       -0.843112         1.593219
    2.336108    0.446136 5.236314 0.000000        1.460636         3.211580
--------------------------------------------------------------------------------
R-squared: 0.224535
Adjusted R-squared: 0.223757
F-statistic: 55.129109
Root MSE: 13.926907
None
